In [1]:
import boto3

In [15]:
default_region='us-east-1'

emr_connection = boto3.client('emr', region_name=default_region)

for cluster in emr_connection.list_clusters()["Clusters"]:
    if cluster["Name"] == "equipo_05" and \
        (cluster["Status"]["State"] == "WAITING" or \
        cluster["Status"]["State"] == "RUNNING"): 
        print("Cluster ID=" + cluster["Id"] + " " + cluster["Status"]["State"])
        jobflow_id = cluster["Id"]

print(emr_connection.list_clusters()["Clusters"])

Cluster ID=j-NMZWAXEMJCJL RUNNING
[{'Id': 'j-NMZWAXEMJCJL', 'Name': 'equipo_05', 'Status': {'State': 'RUNNING', 'StateChangeReason': {'Message': 'Running step'}, 'Timeline': {'CreationDateTime': datetime.datetime(2018, 4, 30, 12, 53, 18, 533000, tzinfo=tzlocal()), 'ReadyDateTime': datetime.datetime(2018, 4, 30, 12, 59, 47, 914000, tzinfo=tzlocal())}}, 'NormalizedInstanceHours': 0}, {'Id': 'j-243VYS5W1T8DH', 'Name': 'equipo_05', 'Status': {'State': 'TERMINATED', 'StateChangeReason': {'Code': 'USER_REQUEST', 'Message': 'Terminated by user request'}, 'Timeline': {'CreationDateTime': datetime.datetime(2018, 4, 29, 22, 25, 46, 709000, tzinfo=tzlocal()), 'ReadyDateTime': datetime.datetime(2018, 4, 29, 22, 31, 45, 633000, tzinfo=tzlocal()), 'EndDateTime': datetime.datetime(2018, 4, 30, 1, 32, 48, 701000, tzinfo=tzlocal())}}, 'NormalizedInstanceHours': 72}, {'Id': 'j-2TC2G7Z9C36QT', 'Name': 'cluster_tarea_8_02', 'Status': {'State': 'TERMINATED_WITH_ERRORS', 'StateChangeReason': {'Code': 'STEP_

In [104]:
# aws emr add-steps --cluster-id JobFlowId 
# --steps Type=spark,Name=parqueteo,
# Args=[--deploy-mode,cluster,--master,yarn,--conf,spark.yarn.submit.waitAppCompletion=true,
# s3a://jorge-altamirano/profeco/parquet.j3a.py],
# ActionOnFailure=CONTINUE
emr_connection.add_job_flow_steps(JobFlowId=jobflow_id,                
                                                         Steps=[{
                                                             'Name': 'parqueteo',
                                                             'ActionOnFailure': 'CONTINUE',
                                                             'HadoopJarStep': {
                                                                 'Jar': 'command-runner.jar',
                                                                 'Args': [
                                                                     "spark-submit",
                   
                                                                     "s3a://jorge-altamirano/profeco/parquet.j3a.py",
                                                                 ]
                                                             }
                                                         }]
                                                        )

Resetting dropped connection: elasticmapreduce.us-east-1.amazonaws.com


{'ResponseMetadata': {'HTTPHeaders': {'content-length': '31',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 30 Apr 2018 04:38:30 GMT',
   'x-amzn-requestid': '557e93ef-4c30-11e8-a3b2-c7ffdc067abb'},
  'HTTPStatusCode': 200,
  'RequestId': '557e93ef-4c30-11e8-a3b2-c7ffdc067abb',
  'RetryAttempts': 0},
 'StepIds': ['s-2JK4ZV2ENOSK0']}

In [76]:
emr_connection.add_job_flow_steps(JobFlowId=jobflow_id,                
                                                         Steps=[{
                                                             'Name': 'sumarizacion-2223',
                                                             'ActionOnFailure': 'CONTINUE',
                                                             'HadoopJarStep': {
                                                                 'Jar': 'command-runner.jar',
                                                                 'Args': [
                                                                     "spark-submit",
                                                                     "s3a://jorge-altamirano/profeco/agg.j3a.py",
                                                                 ]
                                                             }
                                                         }]
                                                        )

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '31',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 30 Apr 2018 22:48:14 GMT',
   'x-amzn-requestid': '913156c2-4cc8-11e8-8bc4-43b7e6877fff'},
  'HTTPStatusCode': 200,
  'RequestId': '913156c2-4cc8-11e8-8bc4-43b7e6877fff',
  'RetryAttempts': 0},
 'StepIds': ['s-2023C7PF1KHIJ']}

In [16]:
for step in emr_connection.list_steps(ClusterId=jobflow_id)["Steps"]:
    if( ((step["Name"] == "sumarizacion" or step["Name"] == "sumarizacion-2223") and \
          step["Status"]["State"] == "COMPLETED") or \
        (step["Name"] == "parqueteo" and step["Status"]["State"] == "COMPLETED")) :
        print( " " + step["Status"]["State"])
        print(step)

In [57]:
jobflow_id = "j-3WEDNH37EGG3"

In [65]:
def parqueteo_completed(jobflow_id): 
    for step in emr_connection.list_steps(ClusterId=jobflow_id)["Steps"]:
#         if step["Name"] == "parqueteo":
#             print(step["Status"]["State"])
        if step["Name"] == "parqueteo" and step["Status"]["State"] == "COMPLETED":
            return True
        else:
            continue
    return False
def sumarizacion_completed(jobflow_id): 
    for step in emr_connection.list_steps(ClusterId=jobflow_id)["Steps"]:
#         if step["Name"] == "sumarizacion" or step["Name"] == "sumarizacion-2223":
#             print(step["Status"]["State"])
        if (step["Name"] == "sumarizacion" or step["Name"] == "sumarizacion-2223") and \
          step["Status"]["State"] == "COMPLETED":
            return True
        else:
            continue
    return False
print("parqueteo " + str(parqueteo_completed(jobflow_id)))
print("sumarizacion " + str(sumarizacion_completed(jobflow_id)))

parqueteo True
sumarizacion False


In [63]:
    def parqueteo_pending(jobflow_id): 
        emr_connection = boto3.client('emr', region_name=default_region)
        for step in emr_connection.list_steps(ClusterId=jobflow_id)["Steps"]:
            if step["Name"] == "parqueteo" and (\
                step["Status"]["State"] == "RUNNING" or step["Status"]["State"] == "PENDING") :
                    return True
            else:
                continue
        return False 
    def sumarizacion_pending(jobflow_id): 
        emr_connection = boto3.client('emr', region_name=default_region)
        for step in emr_connection.list_steps(ClusterId=jobflow_id)["Steps"]:
            if (step["Name"] == "sumarizacion" or step["Name"] == "sumarizacion-2223") and (\
                step["Status"]["State"] == "RUNNING" or step["Status"]["State"] == "PENDING") :
                    return True
            else:
                continue
        return False 
print("parqueteo " + str(parqueteo_pending(jobflow_id)))
print("sumarizacion " + str(sumarizacion_pending(jobflow_id)))

parqueteo False
sumarizacion True


In [18]:
cluster_state = "PENDING"
for cluster in emr_connection.list_clusters()["Clusters"]:
    if cluster["Id"] == jobflow_id:
        cluster_state = cluster["Status"]["State"]
cluster_state

'RUNNING'

In [66]:
for cluster in emr_connection.list_clusters()["Clusters"]:
     if cluster["Id"] == jobflow_id:
        print(cluster)

{'Id': 'j-3WEDNH37EGG3', 'Name': 'equipo_05', 'Status': {'State': 'WAITING', 'StateChangeReason': {'Message': 'Cluster ready after last step completed.'}, 'Timeline': {'CreationDateTime': datetime.datetime(2018, 4, 30, 15, 5, 9, 756000, tzinfo=tzlocal()), 'ReadyDateTime': datetime.datetime(2018, 4, 30, 15, 12, 45, 704000, tzinfo=tzlocal())}}, 'NormalizedInstanceHours': 48}


In [67]:
# emr_connection.terminate_job_flows(JobFlowIds=[jobflow_id])
# emr_connection.cancel_steps(ClusterId=jobflow_id)
jobflow_id

'j-3WEDNH37EGG3'

In [83]:
instancias = list()
for instance in emr_connection.list_instances(ClusterId=jobflow_id)["Instances"]:
    instancias.append(instance["Ec2InstanceId"])
ec2 = boto3.client("ec2")
print("Finalizando instancias:\n%s"%str(instancias))
ec2.terminate_instances(InstanceIds=instancias)

Finalizando instancias:
['i-0558975d34f989b6d', 'i-0629f1c3ed208e518', 'i-05c43b01b8de9ac32']


{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Mon, 30 Apr 2018 22:53:55 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'bb007848-4aa6-442f-8b01-9f03155f7f71',
  'RetryAttempts': 0},
 'TerminatingInstances': [{'CurrentState': {'Code': 48, 'Name': 'terminated'},
   'InstanceId': 'i-05c43b01b8de9ac32',
   'PreviousState': {'Code': 48, 'Name': 'terminated'}},
  {'CurrentState': {'Code': 48, 'Name': 'terminated'},
   'InstanceId': 'i-0558975d34f989b6d',
   'PreviousState': {'Code': 48, 'Name': 'terminated'}},
  {'CurrentState': {'Code': 48, 'Name': 'terminated'},
   'InstanceId': 'i-0629f1c3ed208e518',
   'PreviousState': {'Code': 48, 'Name': 'terminated'}}]}

In [89]:
for cluster in emr_connection.list_clusters()["Clusters"]:
    print(cluster["Status"]["State"])

TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED_WITH_ERRORS
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED
TERMINATED


In [85]:
steps = list()
for step in emr_connection.list_steps(ClusterId=jobflow_id)["Steps"]:
    if step["Status"]["State"] == "RUNNING" or step["Status"]["State"] == "PENDING":
        steps.append(step["Id"])
        
len(steps)

0

In [ ]:
print()